In [1]:
from torchvision.models import ResNet
from torchvision.models.resnet import BasicBlock
import torch.nn as nn
import torch

In [30]:
model_ft = ResNet(BasicBlock, [2, 2, 2, 2])
base_model = nn.Sequential(*list(model_ft.children())[:-2])

gru1 = nn.GRU(32, 64)
gru2 = nn.GRU(64, 128)

fc = nn.Sequential(nn.Dropout(0.5),
                                 nn.Linear(256, 63))

loss = nn.CTCLoss(blank=62)



In [31]:
x = torch.randn(32, 3, 40, 100)
target = torch.randint(low=0, high=63, size=(32, 4), dtype=torch.long)
x = base_model(x)
print(x.shape)
x = nn.AdaptiveAvgPool2d((1))(x)
# x = torch.reshape(x, (-1, 512, 1))
x = torch.reshape(x, (-1, 16, 32))
x1, _ = gru1(x)
x2, _ = gru1(x)
x = x1 + x2
print(x.shape)
x3, _ = gru2(x)
x4, _ = gru2(x)
x = torch.concat((x3, x4), dim=2)
print(x.shape)
x = fc(x)
print(x.shape)
x = torch.softmax(x, dim=2)
print(x.shape)
x = torch.permute(x, (1,0,2))
input_lengths = torch.full(size=(32,), fill_value=16, dtype=torch.long)
target_lengths = torch.full(size=(32,), fill_value=4, dtype=torch.long)
print(input_lengths.shape)
print(target_lengths.shape)
_ = loss(x, target, input_lengths, target_lengths)

torch.Size([32, 512, 2, 4])
torch.Size([32, 16, 64])
torch.Size([32, 16, 256])
torch.Size([32, 16, 63])
torch.Size([32, 16, 63])
torch.Size([32])
torch.Size([32])


In [29]:
target_lengths



tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4])